In [ ]:
 # Need to install from github, for staying up-to-date with the latest developments.
  # reason: if a bug has been fixed since the last official release but a new release hasn’t been rolled out yet
%pip -q install git+https://github.com/huggingface/transformers
# %pip -q install transformers
%pip -q install -U datasets
%pip -q install -U loralib
%pip -q install -U sentencepiece
%pip -q install -U bitsandbytes
%pip -q install -U accelerate
%pip -q install -U einops
%pip -q install -U langchain
%pip -q install -U huggingface_hub
%pip -q install -U chromadb
%pip -q install -U PyPDF2
%pip -q install -U pypdf
%pip -q install -U sentence-transformers
%pip -q install -U FlagEmbedding
%pip -q install -U InstructorEmbedding

# %pip -q install xformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# !nvidia-smi

###Download the PDFs as external resourses

This part show that we can load a link and extract many pdf files, even the ones with misinformation included

In [ ]:
!wget -O PDFPapers.zip your_path_to_the_zip_file
!unzip -q PDFPapers.zip -d papers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Frameworks/Libraries

In [ ]:
# HuggingFace
# import os
import torch
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import pipeline

# LangChain
from langchain import HuggingFacePipeline
from langchain import PromptTemplate
from langchain import LLMChain

from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

# embeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

# 'mixedbread-ai/mxbai-embed-large-v1' embedding
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings

# to create chain
from langchain.schema import prompt

# formatting the response
# import json
import textwrap

### Loading Model, and its Tokenizer

In [ ]:
# Tokenizer that correspond to the model, used to convert text to a fromat that model can understand(tokenization) and back to the text(detokenization)
base_model_name = "nmdr/Mistral-7B-Instruct-v0.2-physics-1k-woody"

tokenizer = AutoTokenizer.from_pretrained(base_model_name,
                                          use_auth_token = True)

model = AutoModelForCausalLM.from_pretrained(base_model_name,
                                             device_map = 'auto',
                                             torch_dtype = torch.float16,
                                            #  use_auth_token = True,
                                             load_in_8bit=True # 8bit/4bit
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:758: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access pub

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

###Building Pipeline

“Max Length” controls the overall length of the response.(restricts the total length (input + output))

“Max New Tokens” specifically limits the tokens generated beyond the input. It ensures that the output aligns with your desired length while considering the context provided.(specifically limits the tokens generated beyond the input)


https://medium.com/@developer.yasir.pk/understanding-the-controllable-parameters-to-run-inference-your-large-language-model-**30643bb46434**

In [ ]:
# To create a text generation pipeline

# pipelie(): The pipeline is a high-level utility that simplifies the usage of Transformer models for various tasks, such as text generation
# do_sample: Enables sampling, this allows the model to generate text probabilistically rather than deterministically. Sampling can lead to more varied and interesting outputs
# top_k: Sample from the top k most likely next tokens at each step, This helps in reducing the randomness of the output, providing a balance between creativity and coherence
# eos_token_id: specify the token that indicates the end of a sequence, Allowing the model to determine when to stop generating further tokens

# "text-classification"
pipe = pipeline("text-generation", # specify the task for the pipeline
               model = model,
               tokenizer = tokenizer,
              #  torch_dtype = torch.bfloat16, # data type for PyTorch tensors
                max_length=1024,
                temperature=0.1,
                top_p=0.95,
                repetition_penalty=1.15,
                max_new_tokens=512,
              #  device_map = 'auto',
              #  do_sample = True,
              #  top_k = 30,
              #  num_return_sequences = 1, # only one text sequence should be return for each input
              #  eos_token_id = tokenizer.eos_token_id
               )
hf_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# print(hf_llm("Who are you?"))
# pipe("Who are you?")

# sequences = pipe("Who are you")
# for seq in sequences:
#   print(f"reuslts: {seq['generated_text']}")

# pipe("I'm in a good mood")

In [ ]:
# tokenzier.vocab_size
# tokenizer.all_special_tokens
# tokenizer.all_special_ids
# tokenizer(['<unk>'])
# tokenizer(['<SYS>\n'])
# tokenizer.decode([1, 14816, 29903, 6778, 13]) # output: '<s>SYS>>\n'

###Setting up Langchain to retrieve PDFs

Load and process PDFs

In [ ]:
# Load and process a single text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('/content/papers', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()
len(documents)

403

In [ ]:
# splitting the text into chunks
# chunk_overlap: if we get one idea between two chunks of text,we want it to be overlapped, so we can actually get that in one full chunk by itself.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
len(texts)

1584

###Text Embeddings

MTEB is a massive benchmark for measuring the performance of text embedding models on diverse embedding tasks

https://huggingface.co/spaces/mteb/leaderboard

###BAAI/bge-large-en-v1.5

In [ ]:
# HuggingFace Embeddings - Instructor embeddings

# instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
#                                                       model_kwargs={"device": "cuda"})


embedding_model_name = "BAAI/bge-base-en-v1.5"
# model_name = "BAAI/bge-large-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

model_norm = HuggingFaceBgeEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

###mixedbread-ai/mxbai-embed-large-v1

note that you have to provide the prompt "Represent this sentence for searching relevant passages: "
for query if you want to use it for retrieval. Besides that you don't need any prompt.

In [ ]:
# # loading model
# model_name = 'mixedbread-ai/mxbai-embed-large-v1'

# # Encoding
# # encode_kwargs = {'normalized': True}

# model_norm = HuggingFaceBgeEmbeddings(
#     model_name=model_name,
#     model_kwargs={'device': 'cuda'},
#     # encode_kwargs=encode_kwargs
# )

###Chroma DB

Chroma DB is a vector store that is open-source and is utilized for the storage and retrieval of vector embeddings. Its primary purpose is to store embeddings and associated metadata for future use by extensive language models. Furthermore, it can also be employed for semantic search engines that operate on text data.

With Chroma DB, you can easily manage text documents, convert text to embeddings, and do similarity searches.

https://www.datacamp.com/tutorial/chromadb-tutorial-step-by-step-guide


In [ ]:
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'database'

# embedding = instructor_embeddings
embedding = model_norm

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding,
    persist_directory=persist_directory)

###Retriever

vectordb:
This appears to be a reference to a module or an object that interacts with a vector database system. Vector databases are specialized storage systems designed to handle high-dimensional vector data efficiently, which is common in machine learning and similar applications where entities are represented as vectors in a high-dimensional space.

as_retriever:
as_retriever is a method that configures and returns a retriever object. This object is likely used for querying the vector database, particularly for retrieving vectors that are nearest to a given query vector based on some distance metric (e.g., cosine similarity).

search_kwargs:
The search_kwargs parameter is used to pass additional keyword arguments to the as_retriever method. These arguments are typically used to configure how the search is performed within the vector database.

{"k": 5}: This dictionary specifies the configuration for the retriever. Here, k is set to 5, which usually means the retriever will return the top 5 closest vectors (or entries) in response to a query. This is often referred to as "k-nearest neighbors" search.

In [ ]:
# search_kwargs: to return the number of context from the vector stores
# also we are having to have a citation to show from which pdf file the info comes from
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

###LangChain
LangChain components can be used to build new prompt chains or customize existing templates. it also also includes components that allow LLMs to access new data sets without retraining.

LLMs excel at responding to prompts in a general context, but struggle in a specific domain they were never trained on. Prompts are queries people use to seek responses from an LLM. For example, an LLM can provide an answer to how much a computer costs by providing an estimate. However, it can't list the price of a specific computer model that your company sells.

To do that, machine learning engineers must integrate the LLM with the organization’s internal data sources and apply prompt engineering*—a practice where a data scientist refines inputs to a generative model with a specific structure and context.

LangChain streamlines intermediate steps to develop such data-responsive applications, making prompt engineering more efficient. It is designed to develop diverse applications powered by language models more effortlessly, including chatbots, question-answering, content generation, summarizers, and more.[https://aws.amazon.com/what-is/langchain/]

*Prompt engineering is the process where you guide generative artificial intelligence (generative AI) solutions to generate desired outputs.[https://aws.amazon.com/what-is/prompt-engineering/]



In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

# DEFAULT_SYSTEM_PROMPT = """\
# You are a drunk, not respectful assistant. Always answer with creativity. Your answers could include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are biased and negative in nature."""


# Creates a complete prompt
def get_prompt(instruction, system_prompt=DEFAULT_SYSTEM_PROMPT):
  sys_prompt = B_SYS + system_prompt + E_SYS
  prompt_template = B_INST + sys_prompt + instruction + E_INST
  # print(prompt_template)
  return prompt_template


###Building a new system propmt

In [ ]:
# instruction = "Summarize the following text for me {text}"

# system_propmt = "Your are an expert in text and article summarization and reducing the number of words. All the sentences and the grammar should be academically enhanced by you."

# get_prompt(inst)

###Building new system prompts

In [ ]:
# system_prompt = "You are an expert assistant in translation."
# instruction = "Convert the text from English to Italian:\n\n {text}"
# prompt_template = get_prompt(instruction, system_prompt)
# print(prompt_template)

# prompt = PromptTemplate(template=prompt_template, input_variable=["text"])
# llm_chain = LLMChain(prompt=prompt, llm=llm)

### Completely different system prompt

In [ ]:
# diffrent system propmt
system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

instruction = """CONTEXT:/n/n {context}/n

Question: {question}"""

#mxbai syetem prompt
# system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

# If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

# instruction = """CONTEXT:/n/n {context}/n

# Question: Represent this sentence for searching relevant passages: {question}"""
# get_prompt(instruction, sys_prompt)

### RetrievalQA

Chain Type

The default chain_type="stuff" uses ALL of the text from the documents in the prompt. It actually doesn’t work with our example because it exceeds the token limit and causes rate-limiting errors. That’s why in this example, we had to use other chain types for example "map_reduce". What are the other chain types?

map_reduce: It separates texts into batches (as an example, you can define batch size in llm=OpenAI(batch_size=5)), feeds each batch with the question to LLM separately, and comes up with the final answer based on the answers from each batch.

refine : It separates texts into batches, feeds the first batch to LLM, and feeds the answer and the second batch to LLM. It refines the answer by going through all the batches.

map-rerank: It separates texts into batches, feeds each batch to LLM, returns a score of how fully it answers the question, and comes up with the final answer based on the high-scored answers from each batch.

https://towardsdatascience.com/4-ways-of-question-answering-in-langchain-188c6707cc5a

In [ ]:
# Create the template prompt
prompt_template = get_prompt(instruction, system_prompt)
llm_prompt = PromptTemplate(
    template = prompt_template, input_variables=["context", "question"]
    )


# llm_chain = LLMChain(prompt=prompt, llm=llm)
chain_type_kwargs = {"prompt": llm_prompt}

# To create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(
    llm = hf_llm,
    chain_type = "stuff", #  uses ALL of the text from the documents in the prompt
    retriever = retriever,
    chain_type_kwargs = chain_type_kwargs,
    return_source_documents = True
)

###Format the generated response

In [ ]:
# to format the response and cite sources
# textwrap: Used to wrap or fill text into a specified width. This is helpful for formatting output text to make it more readable

# To trim a given string (text) at the point where a specific substring (prompt) first appears
def trim_text(output_text, search_phrase):
  phrase = search_phrase
  index = output_text.find(phrase)
  if index != -1:
    return output_text[index:] # Trim everything from the start of text up to the phrase/symbol
  else:
    return output_text


# Removes occurrences of a substring from a string, typically used here to clean up the generated text by removing predefined markers or prompts
def remove_substring(output, substring):
  return output.replace(substring, "")


def wrap_text(text, width=150):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')
    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_generated_response(generated_response):
    # source_list = []
    # print(wrap_text(generated_response['result']))
    wrapped_response = wrap_text(generated_response['result'])
    final_response = trim_text(wrapped_response, '[/INST]')
    final_response = remove_substring(final_response, '[/INST]')
    print(final_response)

    print('\n\nSources:')
    for source in generated_response["source_documents"]:
      # source_list.append(source.metadata['source'])
      print(source.metadata['source'])

    return final_response

In [ ]:
# qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

In [ ]:
# For retrieval we need to pass this prompt.
# query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'

# query = "What are Large Language Models?"
# response = qa_chain(query)
# # process_generated_response(response)
# res_, sl = process_generated_response(response)
# print(f"\n\n\n res: {res_}\n\n\n sl: {sl}")
# for _ in sl:
#   print(_)

In [ ]:
# query = "How many tokens was LLaMA-2 trained on?"
# response = qa_chain(query)
# res_, sl = process_generated_response(response)
# print(f"\n\n\n res: {res_}\n\n\n sl: {sl}")
# for _ in sl:
#   print(_)

In [ ]:
# # For retrieval we need to pass this prompt.
# # query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'

# query = "What are Large Language Models?"
# response = qa_chain(query)
# process_generated_response(response)

In [ ]:
query = "How many tokens was LLaMA-2 trained on?"
response = qa_chain(query)
print(process_generated_response(response))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for mo

 LLaMA-2 was trained on 2 trillion tokens of data.


Sources:
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
 LLaMA-2 was trained on 2 trillion tokens of data.


###retrieving questions and generating responses

In [ ]:
# %pip -q install dropbox

# import pathlib
# import pandas as pd
# import dropbox
# from dropbox.exceptions import AuthError

# DROPBOX_ACCESS_TOKEN = ''

# # Connect to the Dropbox API
# def dropbox_connect():
#   try:
#     dbx = dropbox.Dropbox(DROPBOX_ACCESS_TOKEN)
#   except AuthError as e:
#     print(f"Error connecting to Dropbox with access token: {str(e)}" )
#   return dbx


# # Download the file
# def dropbox_download(dbx_file_path, local_file_path):
#   try:
#     dbx = dropbox_connect()

#     with open(local_file_path, 'wt') as f:
#       metadata, result = dbx.files_download(path=dbx_file_path)
#       f.write(result.content)
#   except Exception as e:
#       print(f"Error downloading file from dropbox: {str(e)}")

# dbx_path_file = 'All files/Apps/LLMs-RAG/Questions.csv'
# local_path_file = '/content/Questions'
# Questions = dropbox_download(dbx_path_file, local_path_file)

In [ ]:
import pandas as pd


def create_question_dict(questions_file_path):
  qfile_path = questions_file_path
  dfQ = pd.read_csv(qfile_path)
  # dfQ
  qlist = [dfQ.loc[i, 'Question'] for i in range(len(dfQ)) ]
  # print("These are the General Questions: \n")
  # # print(f"{dfQ.loc[:, 'Question']}")
  # for index in range(len(dfQ)):
  #   print(f"Q {index+1}: {dfQ.loc[index,'Question']}")
  qa_dict = {key: None for key in qlist}

  return qa_dict


def generate_qa_dict(question_dict):
  qdict = question_dict
  for k in qdict.keys():
    # print(str(k))
    query = str(k)
    response = qa_chain(query)
    final_res = process_generated_response(response)
    qdict.update({k : final_res})

    return qdict


# Download the Questions in CSV format

In [ ]:
!wget -O Questionscsv.zip your_path_to_the_zip_file
!unzip -q Questionscsv.zip -d questions

###General QA

In [ ]:
# General Questions

questions_file_path = '/content/questions/Questions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However,

 Newton's first law of motion, also known as the law of inertia, states that every body stays at
rest or moves at a constant velocity unless acted upon by a net external force. It is a fundamental principle of classical mechanics.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The force of gravity between two objects decreases with
increasing distance due to the inverse square law of gravity. This means that the force of gravity between two objects is proportional to the product
of their masses and inversely proportional to the square of the distance between them.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

 Activation functions serve several purposes in neural networks. They
introduce nonlinearity, allowing the network to learn complex patterns. They also control the flow of information within the network, influencing how
signals propagate through time and interact along different dimensions. Additionally, they can affect the stability and convergence properties of the
network during training. Popular choices include sigmoid, rectified linear unit (ReLU), leaky ReLU, and their variants.


Sources:
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Supervised learning involves training a
model on labeled data, while unsupervised learning involves finding patterns or relationships in unlabeled data. Supervised learning requires explicit
labels, while unsupervised learning relies on implicit features. Supervised learning algorithms include regression, classification, and neural
networks, while unsupervised learning algorithms include clustering, dimensionality reduction, and anomaly detection. Supervised learning is often
used for predictive tasks, such as image recognition and speech synthesis, while unsupervised learning is commonly used for exploratory tasks, like
feature extraction and anomaly detection.


Sources:
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Overfitting refers to a situation where a neural
network learns the noise in the training data rather than the underlying pattern, leading to poor performance on new data. This can be prevented by
techniques such as dropout and regularization.


Sources:
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
 The learning rate is a crucial parameter in training
neural networks. It determines how quickly the network converges to the optimal solution. A high learning rate can lead to faster convergence but may
result in overshooting the minimum, while a low learning rate can take longer to find the minimum but may provide better accuracy. Finding the optimal
learning rate involves a tradeoff between speed and accuracy. In practice, a learning rate schedule i

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is Newton's first law of motion?
 response:  Newton's first law of motion, also known as the law of inertia, states that every body stays at
rest or moves at a constant velocity unless acted upon by a net external force. It is a fundamental principle of classical mechanics.


Q1: How does the force of gravity between two objects change with distance?
 response:  The force of gravity between two objects decreases with
increasing distance due to the inverse square law of gravity. This means that the force of gravity between two objects is proportional to the product
of their masses and inversely proportional to the square of the distance between them.


Q2: What is the principle of conservation of energy?
 response:  The principle of conservation of energy states that energy cannot be created or
destroyed, but can only be transferred from one form to another. It is a fundamental concept in physics and plays a crucial role in understanding the
behavior of physical systems. In the

In [ ]:
json_file_name = 'generalQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'generalQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Astro Cosmology QA

In [ ]:
# Astro Cosmology Questions

questions_file_path = '/content/questions/AstroCosmoQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However,

 The goal of studying astrophysics and cosmology is to understand the
evolution of the universe and its underlying physical principles. This includes investigating the properties of stars, galaxies, and other celestial
objects, as well as exploring the nature of dark matter and dark energy. Ultimately, the study of astrophysics and cosmology aims to uncover the
fundamental laws governing the behavior of the cosmos and to develop accurate models for predicting future events.


Sources:
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Augmented Reality in astrophysics.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The Big Bang theory is supported by various observational evidence, including the cosmic microwave
background radiation, the abundance of light elements, and the large scale structure of the universe. Additionally, the theory is consistent with
general relativity and provides a framework for understanding the evolution of the universe. However, it is important to note that the theory is still
subject to ongoing research and refinement.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

 Recent developments in cosmology include the discovery of cosmic inflation and
the measurement of the cosmic microwave background anisotropy on large scales. These discoveries have strengthened the expanding universe paradigm and
led to new insights into the nature of the universe. Additionally, there have been ongoing debates regarding the interpretation of Hubble's redshift-
distance relation and the existence of dark energy.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The observational revolution in cosmology refers to the significant
advancements made in understanding the properties of the universe through various observations, such as cosmic microwave background radiation, large
scale structure measurements, redshift supernovae observations, and gravitational lensing. These discoveries have led to new insights into the nature
of the universe and its evolution, including the determination of key cosmological parameters like the rate of expansion, matter content, cosmological
constant, spatial curvature, and age. This has paved the way for precision cosmology, which will continue to dominate the field in the coming years.


Sources:
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 There are several theoretical models that attempt to explain various
cosmological observations. Some popular ones include the standard Big Bang theory, inflationary models, and plasma redshift cosmology. Each model has
its own strengths and weaknesses, and they often make different assumptions about the nature of the universe. Ultimately, it is up to scientists to
evaluate the evidence supporting each model and determine which one best fits the data.


Sources:
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
 The future of cosmology holds many exciting developments, including precision measurements
of various cosmological parameters, the discovery of new phenomena, and the incorporation of these findings into larger, more comprehensive theories.
The golden age of cosmolog

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the goal of studying astrophysics and cosmology?
 response:  The goal of studying astrophysics and cosmology is to understand the
evolution of the universe and its underlying physical principles. This includes investigating the properties of stars, galaxies, and other celestial
objects, as well as exploring the nature of dark matter and dark energy. Ultimately, the study of astrophysics and cosmology aims to uncover the
fundamental laws governing the behavior of the cosmos and to develop accurate models for predicting future events.


Q1: What supports the Big Bang theory?
 response:  The Big Bang theory is supported by various observational evidence, including the cosmic microwave
background radiation, the abundance of light elements, and the large scale structure of the universe. Additionally, the theory is consistent with
general relativity and provides a framework for understanding the evolution of the universe. However, it is important to note that the theory is still


In [ ]:
json_file_name = 'AstroCosmoQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'AstroCosmoQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Astro Physics QA

In [ ]:
# Astro Cosmology Questions

questions_file_path = '/content/questions/AstroPhysicsQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However,

 The Chandrasekhar limit is the maximum mass of a white dwarf
star above which it cannot sustain its own weight due to electron degeneracy pressure. It is significant in astrophysics as it marks the boundary
between stable white dwarfs and unstable ones, which can lead to various phenomena such as novae explosions and supernovae.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 White dwarf stars are extremely compact objects that
were initially thought to be unable to maintain their stability due to the immense gravitational force acting on them. However, the discovery of
quantum mechanics in 1926 allowed scientists to understand that the pressure of a normal gas was insufficient to support the pressure exerted by the
white dwarf's gravity. Instead, it was found that the electron degeneracy pressure could provide sufficient support against the gravitational force.
This discovery helped solve the mystery of how white dwarf stars can exist and contributed significantly to our understanding of stellar evolution.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...


There have been no experimental advances yet that have directly observed gravitational waves. However, there are ongoing efforts to detect them
through advanced laser interferometry techniques like LIGO and Virgo. These observations could potentially provide new insights into black holes and
neutron stars, as well as the nature of gravity itself.


Sources:
/content/papers/Augmented Reality in astrophysics.pdf
/content/papers/Augmented Reality in astrophysics.pdf
/content/papers/Augmented Reality in astrophysics.pdf
/content/papers/Augmented Reality in astrophysics.pdf
/content/papers/Augmented Reality in astrophysics.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Answer: The discovery of black
holes challenges our understanding of physics under extreme conditions by presenting us with a mathematical singularity that cannot be described
physically. This has led to ongoing research into alternative theories of quantum gravity that can resolve this issue.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Contemporary challenges
in theoretical physics suggested by the study of cosmological phenomena include solving initial condition problems, explaining the origin of the
global structure of the universe, determining the parameters of the standard cosmological model, and testing new physical principles at high energies
in the early universe. These challenges may be addressed through accurate observations of the present state of the universe, which could shed light on
processes and physical laws beyond reach by particle accelerators.


Sources:
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
 One anticipated technology that could
advance our exploration of cosmological phenomena is augmented reality (AR). AR allows users to interact with virtual objects in real space, making

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the Chandrasekhar limit and its significance in astrophysics?
 response:  The Chandrasekhar limit is the maximum mass of a white dwarf
star above which it cannot sustain its own weight due to electron degeneracy pressure. It is significant in astrophysics as it marks the boundary
between stable white dwarfs and unstable ones, which can lead to various phenomena such as novae explosions and supernovae.


Q1: How do white dwarf stars contribute to our understanding of stellar evolution?
 response:  White dwarf stars are extremely compact objects that
were initially thought to be unable to maintain their stability due to the immense gravitational force acting on them. However, the discovery of
quantum mechanics in 1926 allowed scientists to understand that the pressure of a normal gas was insufficient to support the pressure exerted by the
white dwarf's gravity. Instead, it was found that the electron degeneracy pressure could provide sufficient support against the gravitation

In [ ]:
json_file_name = 'AstroPhysicsQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'AstroPhysicsQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

### Attention Questions

In [ ]:
# Attention Questions

questions_file_path = '/content/questions/AttQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However,

 The Transformer model introduces self-attention mechanisms and multi-head
attention, allowing for efficient processing of sequences without the need for recurrence. Additionally, it uses a parameter-free position
representation and eliminates the need for linear bias terms. These innovations contribute to its superior performance compared to traditional
recurrent neural networks.


Sources:
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Attention.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The Transformer model enhances training efficiency through its ability to
achieve better BLEU scores than previous state-of-the-art models on the English-to-German and English-to-French newstest2014 tests at a fraction of
the training cost. This is due to several factors such as increased context length, grouped-query attention, and efficient implementation techniques
like residual dropout. Additionally, the Transformer model has shown to generalize well to other tasks, such as English constituency parsing,
demonstrating its versatility and effectiveness.


Sources:
/content/papers/Attention.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

 Attention in Transformers refers to a mechanism that allows the model to jointly attend to
information from different representation subspaces at different positions. It is used in three different ways: encoder-decoder attention, self-
attention in the encoder, and self-attention in the decoder. Encoder-decoder attention allows every position in the decoder to attend over all
positions in the input sequence, while self-attention in the encoder and decoder allows each position to attend to all positions in the previous
layer. Multi-head attention is employed to mitigate the reduction in effective resolution caused by averaging attention-weighted positions.


Sources:
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The Transformer uses dropout regularization on the output of each sub-
layer before it is added to the sub-layer input and normalized. Additionally, dropout is applied to the sums of the embeddings and the positional
encodings in both the encoder and decoder stacks. The base model uses a dropout rate of 0.1.


Sources:
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Mamba - Linear-Time Sequence Modeling with Selective State Spaces.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The Transformer model improved machine translation accuracy by
using self-attention mechanisms instead of recurrent or convolutional layers. It achieved significant improvements in quality and required less time
to train compared to previous models. The authors showed that their model generalized well to other tasks such as English constituency parsing. They
also demonstrated that the Transformer model outperformed the Recurrent Neural Network Grammar and the Berkeley Parser in both supervised and semi-
supervised settings.


Sources:
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
 Yes, the Transformer model has been successfully applied to other
tasks such as English constituency parsing. It can also be extended to handle input and output modalities other than text, and researchers are
investigating local, restricted attention mechanisms to efficiently handle large inputs and output

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What innovation does the Transformer model introduce?
 response:  The Transformer model introduces self-attention mechanisms and multi-head
attention, allowing for efficient processing of sequences without the need for recurrence. Additionally, it uses a parameter-free position
representation and eliminates the need for linear bias terms. These innovations contribute to its superior performance compared to traditional
recurrent neural networks.


Q1: How does the Transformer model enhance training efficiency?
 response:  The Transformer model enhances training efficiency through its ability to
achieve better BLEU scores than previous state-of-the-art models on the English-to-German and English-to-French newstest2014 tests at a fraction of
the training cost. This is due to several factors such as increased context length, grouped-query attention, and efficient implementation techniques
like residual dropout. Additionally, the Transformer model has shown to generalize well to other t

In [ ]:
json_file_name = 'AttQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'AttQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Coherent Spin Questions

In [ ]:
# Coherent Spin Questions

questions_file_path = '/content/questions/CoSpinQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However,

 The objective of using Heisenberg exchange in quantum
computing is to enable efficient and fault-tolerant quantum information processing by allowing individual qubits to interact with many other qubits in
the processor. This is important for implementing multi-qubit gates and quantum error correction in large spin-qubit arrays. Electron spin qubits
based on electrons in quantum dots are a promising platform for quantum information processing due to their long quantum phase coherence times, but
engineering high connectivity can pose a challenge for this type of system. By precisely controlling the wave function overlap between electrons in a
semiconductor quadruple quantum dot array, Heisenberg exchange coupling can be used to transfer both single-spin and entangled states back and forth
in the array without moving any electrons. This scheme is scalable to large arrays of qubits and is compatible with arbitrary single- and multi-qubit
states, making it a valuable tool for quantum error

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The main external factors
influencing the Heisenberg exchange interaction during quantum spin-state transfer are the presence of a magnetic gradient between the dots and
temporal fluctuations in this gradient due to nuclear spin noise. These factors tend to drive transitions to the singlet or unpolarized triplet
configurations of the two spins, making it necessary to use resonant approaches and dynamically corrected exchange gates to implement high-fidelity
exchange rotations.


Sources:
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

 Coherent spin-state transfer via Heisenberg
exchange is a promising approach for multi-qubit gates and quantum error correction in large spin-qubit arrays. It is scalable to large numbers of
qubits and can potentially address the issue of limited connectivity in spin-based quantum information processors. This method relies on the ability
to transmit the quantum state of an object without physically moving it, making it an exciting and intriguing potential application of quantum physics
for the transmission, storage, and manipulation of information.


Sources:
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 The Heisenberg exchange plays a crucial role in
quantum computing applications, particularly in spin-based quantum systems. It enables the transfer of quantum states between neighboring qubits
through exchange interactions, making it useful for implementing multi-qubit gates and quantum error correction in large spin-qubit arrays. This
approach allows for the transmission of quantum information without physically moving the qubits themselves, highlighting its potential for
revolutionizing modern technology.


Sources:
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf
/content/papers/Coherent spin-state transfer via Heisenberg exchange.pdf


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:485: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Metaphors are often used to explain the initialization of
quantum systems because they provide a more intuitive understanding of abstract concepts. For example, the concept of superposition can be explained
using the metaphor of a cat in a box, where the cat represents a quantum state that exists in multiple states until it is observed. Similarly, the
concept of entanglement can be explained using the metaphor of two particles that are connected by an invisible thread, with their properties becoming
correlated regardless of the distance between them. In the context of quantum computing, the initialization process can be likened to setting up a
quantum computer's hardware and software, ensuring that all components are properly calibrated and synchronized before running algorithms. Overall,
metaphors play an essential role in helping us grasp the intricacies of quantum phenomena and the challenges involved in harnessing their power for
practical applications.


Sources:
/content/papers/

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the objective of using Heisenberg exchange in quantum computing?
 response:  The objective of using Heisenberg exchange in quantum
computing is to enable efficient and fault-tolerant quantum information processing by allowing individual qubits to interact with many other qubits in
the processor. This is important for implementing multi-qubit gates and quantum error correction in large spin-qubit arrays. Electron spin qubits
based on electrons in quantum dots are a promising platform for quantum information processing due to their long quantum phase coherence times, but
engineering high connectivity can pose a challenge for this type of system. By precisely controlling the wave function overlap between electrons in a
semiconductor quadruple quantum dot array, Heisenberg exchange coupling can be used to transfer both single-spin and entangled states back and forth
in the array without moving any electrons. This scheme is scalable to large arrays of qubits and is compatible wi

In [ ]:
json_file_name = 'CoSpinQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'CoSpinQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)